<a href="https://colab.research.google.com/github/syunar/22-01_Laptops-Prices-Prediction-Analysis/blob/main/22-01_01_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install selenium and chrimedriver
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 8.1 MB/s 
     |████████████████████████████████| 358 kB 48.7 MB/s 
     |████████████████████████████████| 139 kB 47.2 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 4.1 MB 44.5 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://

In [ ]:
# import libraries
from selenium.webdriver.common.by import By
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
import pandas as pd
import time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# STEP
# 1. get into first page.
#   1.1 find the number and elements like webstructure of each products on 1 page.
#   1.2 collect the data like name, price, url of each products by pointer element from 1.1.
# 2. loop to page 2, 3, ... , n.
# 3. get into each product_urls.
#   3.1 find element of data e.g. CPU, GPU, model_year, size, refresh_rate, etc.
#   3.1 collect data to dict

# n_product = 21 
# pages = int(n_product / 21) 
pages = 112
all_products = [] 

# loop to each pages to acquire datas
for page in range(1,pages+1):
  
  url = f'https://notebookspec.com/notebook/search?page={page}'
  driver.get(url)
  
  n_products = len(driver.find_elements(By.CSS_SELECTOR, '.item-info .title'))
  products = range(1,n_products+1)

  # get into each product div and collect data only inside the div
  for product in products:

    get_names = driver.find_elements(By.CSS_SELECTOR, f'.search-result-item:nth-child({product}) .title')
    get_prices = driver.find_elements(By.CSS_SELECTOR, f'.search-result-item:nth-child({product}) span')
    get_urls = driver.find_elements(By.CSS_SELECTOR, f'.search-result-item:nth-child({product}) .item-info a')
    
    names = [name.get_attribute('innerText').strip() for name in get_names]
    prices = [price.get_attribute('innerText').strip() for price in get_prices]
    urls = [url.get_attribute('href').strip() for url in get_urls]

    print(f'get urls: page={page}, product_count={product}, {names[0]}')

    keys = ['names', 'prices', 'urls'] # combine the keys
    values = names + prices + urls # combine the values

    # convert list of keys and values to dict
    tempdata_dict = {}
    tempdata_dict.update({k:v for k,v in zip(keys,values)})
    all_products.append(tempdata_dict)
  # end for loop products on each page
# end for loop pages


# loop to each products link url to collect more data
  count = 1
for product in all_products:
  url = product['urls']
  driver.get(url)

  get_keys = driver.find_elements(By.CSS_SELECTOR, '.specificate-title')
  get_values = driver.find_elements(By.CSS_SELECTOR, '.specificate-info')
  keys = [key.get_attribute('innerText') for key in get_keys]
  values = [value.get_attribute('innerText') for value in get_values]

  product.update({k:v for k,v in zip(keys,values)})

  print(f'get infos: product_count={count}, {product["names"]}')
  count += 1
# end for loop product_urls

df = pd.DataFrame(all_products)
print(df.info())





get urls: page=1, product_count=1, ['Acer Predator Helios 300 PH317-56-75RR']
get urls: page=1, product_count=2, ['Acer Predator Helios 300 PH317-56-71ZW']
get urls: page=1, product_count=3, ['Acer Nitro 5 AN515-58-705T']
get urls: page=1, product_count=4, ['Acer Nitro 5 AN515-58-50WD']
get urls: page=1, product_count=5, ['Acer Nitro 5 AN515-46-R12A']
get urls: page=1, product_count=6, ['MSI GF63 Thin 11UC-877TH']
get urls: page=1, product_count=7, ['Huawei MateBook 16s-i7/16/1TB']
get urls: page=1, product_count=8, ['Acer Swift 5 SF514-56T-788G']
get urls: page=1, product_count=9, ['Acer Swift 5 SF514-56T-71VH']
get urls: page=1, product_count=10, ['Asus TUF Gaming A15 FA506ICB-HN103W']
get urls: page=1, product_count=11, ['HP Victus Gaming 16-e1081AX']
get urls: page=1, product_count=12, ['HP Victus Gaming 16-e1080AX']
get urls: page=1, product_count=13, ['HP 15s-fq2665TU']
get urls: page=1, product_count=14, ['HP 15s-eq3002AU']
get urls: page=1, product_count=15, ['HP 15s-eq3001AU']

In [ ]:
df.head()

In [ ]:
df.to_csv('nbspec.csv')